In [ ]:
import os
import numpy as np

# 0 empty, 1 walls

In [ ]:
id_instance = 7
while id_instance <= 50:
    rows = np.random.randint(6, 10)
    cols = np.random.randint(6, 10)
    n = cols*rows
    num_crates = np.random.randint(1, 6 if (rows>=10 or cols>=10) else 4)
    
    # Start positions for agent, crates and targets
    pos_agent = (np.random.randint(1, rows-1), np.random.randint(1, cols-1))
    pos_crates, pos_targets = [], []
    walls = [x for x in range(cols)]+[x for x in range(n-cols, rows*cols)]  # Horizontal borders
    walls = walls + [x for x in range(cols, n-cols, cols)]  # Vertical borders
    walls = sorted(walls + [x+cols-1 for x in range(cols, n-cols, cols)])  # Vertical borders
    for id_crate in range(num_crates):
        pos_crate = (np.random.randint(2, rows-2), np.random.randint(2, cols-2))  # avoid blocks near borders
        while pos_crate == pos_agent or pos_crate in pos_crates:  # Avoid sovrappositions
            pos_crate = (np.random.randint(2, rows-2), np.random.randint(2, cols-2))  # avoid blocks near borders
        pos_crates.append(pos_crate)
        pos_target = (np.random.randint(1, rows-1), np.random.randint(1, cols-1))
        while pos_target == pos_crate:  # Avoid easy win
            pos_target = (np.random.randint(1, rows-1), np.random.randint(1, cols-1))
        pos_targets.append(pos_target)
        
    # solve with a relaxed version of the model
    filename = "sokoban_New"+str(id_instance)+".param"
    with open(filename, "w") as f:
        f.write("language ESSENCE' 1.0\n")
        f.write("letting w be "+str(cols)+"\n")
        f.write("letting n be "+str(n)+"\n")
        f.write("letting noCrates be "+str(num_crates)+"\n")
        f.write("letting noGoals be "+str(num_crates)+"\n")
        f.write("letting pInit be "+str((pos_agent[0]*cols)+(pos_agent[1]))+"\n")
        f.write("letting stps be "+str(int(0.2*(rows-1)*(cols-1)))+"\n")
        f.write("letting crates be "+str([pos_crates[i][0]*cols+pos_crates[i][1] for i in range(num_crates)])+"\n")
        f.write("letting goals be "+str([pos_targets[i][0]*cols+pos_targets[i][1] for i in range(num_crates)])+"\n")
        f.write("letting wallsInit be "+str(walls)+"\n")
        f.write("letting noWallsInit be "+str(len(walls))+"\n")
        
        # Service parameters, to be removed in final param file
        f.write("letting upper_bound_changes be "+str(np.random.randint(5, 11))+"\n")
        #f.write("letting upper_bound_steps be "+str(int(0.33*(n-1)*(n-1))))
    
    a = os.system("../../savilerow -timelimit 600 -solver-options '-cpulimit 3600' sokoban_relaxed.eprime %s -run-solver -out-solution s0" % (filename))

    flag = False
    if not os.path.exists(filename+".info"):
        flag = True
    else:
        with open(filename+".info", "r") as f:
            for line in f.readlines():
                if "SolverSolutionsFound:0" in line:
                    flag = True
                    break
    if flag:
        continue
        
    with open("s0", "r") as f:
        lines = f.readlines()
    for line in lines:
        if "noWalls" in line:
            num_walls = int(line.split(" ")[-1])
        if "walls" in line:
            line = line.split("be ")[-1].split(";")[0][1:]
            walls = [int(x.split(",")[0]) for x in line.split(" ") if x != "," and x!=" "]
            walls = [x for x in walls if x>=0]

    with open(filename, "r") as f:
        # Removing Service parameters
        old_file = f.readlines()[:-1]
    os.system("rm "+filename)
    with open(filename, "w") as f:
        for line in old_file:
            if "wallsInit" in line:  # wallsInit and noWallsInit are the last parameters
                break
            f.write(line)
        f.write("letting noWalls be "+str(num_walls)+"\n")
        f.write("letting walls be "+str(walls))
        
    id_instance += 1

#os.system("rm s0")
#os.system("rm .MINION*")
#os.system("rm *.infor")
#os.system("rm *.info")
#os.system("rm *.minion")